In [10]:
from PIL import Image, ImageOps
import requests
import pathlib
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

ModuleNotFoundError: No module named 'distutils'

In [ ]:
url = 'https://0245-71-92-86-252.ngrok-free.app/base.keras'
r = requests.get(url)
open('base.keras', 'wb').write(r.content)

In [3]:
dir_name = input('Enter Data Folder Name: ')
data_dir = pathlib.Path(dir_name)
image_size = (256, 256)
all_images = list(data_dir.glob('*/*.jpg'))

In [4]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

25331


In [5]:
categories = list(data_dir.glob('*/'))

In [ ]:
dataset_dict = {}
os.mkdir('resized')
for (category_index, category) in enumerate(categories):
	os.mkdir(pathlib.Path('resized') / category.relative_to(data_dir))
	category_dir = list(category.glob('*.jpg'))
	dataset_dict[category.name] = []
	for (image_index, image) in enumerate(category_dir):
		with Image.open(image) as img:
			print(f"Working on {image.name} in {category.name} (Image {image_index+1}/{len(category_dir)} of Category {category_index+1}/{len(categories)})")
			resized_img = ImageOps.fit(img, image_size)
			resized_img.save(pathlib.Path('resized') / image.relative_to(data_dir))
	print("Finished!")

In [6]:
data_dir = pathlib.Path(dir_name).parent / 'resized'

In [7]:
batch_size = 32
img_height = 256
img_width = 256

In [8]:
train_ds = tf.keras.utils.image_dataset_from_directory(
	data_dir,
	validation_split=0.2,
	subset="training",
	seed=123,
	image_size=(img_height, img_width),
	batch_size=batch_size
)

Found 25331 files belonging to 8 classes.
Using 20265 files for training.


2024-03-07 00:23:19.378882: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Pro
2024-03-07 00:23:19.378908: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-07 00:23:19.378915: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-07 00:23:19.378966: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-07 00:23:19.379201: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [9]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 25331 files belonging to 8 classes.
Using 5066 files for validation.


In [10]:
class_names = train_ds.class_names
print(class_names)

['Actinic keratosis', 'Basal cell carcinoma', 'Benign keratosis', 'Dermatofibroma', 'Melanocytic nevus', 'Melanoma', 'Squamous cell carcinoma', 'Vascular lesion']


In [11]:
for image_batch, labels_batch in train_ds:
	print(image_batch.shape)
	print(labels_batch.shape)
	break

(32, 256, 256, 3)
(32,)


In [12]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
print(np.min(first_image), np.max(first_image))

In [13]:
num_classes = len(class_names)

model = keras.saving.load_model("base.keras")

In [14]:
model.compile(optimizer='adam',
	loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
	metrics=['accuracy']
)

In [15]:
epochs=10
history = model.fit(
	train_ds,
	validation_data=val_ds,
	epochs=epochs
)

Epoch 1/10


: 

In [ ]:
print(tf.__version__)

NameError: name 'tf' is not defined

In [ ]:
model.save('addition.keras')